# https://www.bogleheads.org/forum/viewtopic.php?p=4837326#p4837326

In [ ]:
%matplotlib inline
import plot
import market
import metrics
from decimal import Decimal
import pandas
import simulate
import withdrawal
import harvesting
import seaborn
import math

In [ ]:
class am_bogleheads(withdrawal.WithdrawalStrategy):
    def __init__(self, portfolio, harvest):
        super().__init__(portfolio, harvest)
        self.floor = self.portfolio.value * Decimal('0.03')

    def start(self):
        return self.portfolio.value * Decimal('.05')

    def next(self):
        adjusted_floor = self.floor * self.portfolio.inflation
        return max(adjusted_floor, self.portfolio.value * Decimal('.05'))


In [ ]:
series = market.Returns_US_1871()

In [ ]:
TYPE = 'Withdrawal (Real)'
m = {'Withdrawal (Nominal)': lambda x: float(x.withdraw_n),
     'Withdrawal (Real)' : lambda x: float(x.withdraw_r),
     'Portfolio (Nominal)': lambda x: float(x.portfolio_n),
     'Portfolio (Real)': lambda x: float(x.portfolio_r),
     'Withdrawal Percent': lambda x: float(x.withdraw_pct_cur)
}
lens = m[TYPE]
lens.__label__ = TYPE

def fn(x):
    return [lens(_) for _ in x]


In [ ]:
year = 1966

w = {
    'vpw': lambda p,h: withdrawal.VPW(p, h, years_left=45),
    'am': am_bogleheads,
    '5%': withdrawal.ConstantPercentage,
}

def run_sim(year, key, length=15):
    portfolio = (600_000, 400_000)
    harvest_strategy = harvesting.make_rebalancer(0.6)
    
    return simulate.withdrawals(series.iter_from(year),
                                 withdraw=w[key],
                                 years=length,
                                 portfolio=portfolio,
                                 harvesting=harvest_strategy)


In [ ]:
def blanchett_smile(age, target=40_000):
    """ This returns the real annual change in consumption at a given age.
    From Blanchett's "Estimating the True Cost of Retirement" (2013)
    """
    spend = (0.00008 * age * age) - (0.0125 * age) - (0.0066 * math.log(target)) + 0.546
    return spend

def spending():
    s = []
    spend = 80_000
    for i in range(65,95):
        delta = blanchett_smile(i, target=80_000)
        spend *= (1 + delta)
        s.append(spend)
    return s

In [ ]:
df = pandas.DataFrame({
    'vpw' : [lens(x) for x in run_sim(year, 'vpw')],
    'am' : [lens(x) for x in run_sim(year, 'am')],
#    'spending' : spending(),
})
g = seaborn.relplot(
        data=df,
        kind='line',
        aspect=2,
    )
#g.fig.autofmt_xdate()
g.despine(left=True, bottom=True, offset=20)
g.fig.suptitle(f'Retirement in {year}')

In [ ]:
df[df['am'] > df['vpw']]

In [ ]:
a = df[df['am'] > df['vpw']]

In [ ]:
a['am'] / a['vpw']

In [ ]:
all_s = {}
for strategy in ['vpw', 'am']:
    s = pandas.Series()
    for year in range(1871, 2018-15):
        withdraws = fn(run_sim(year, strategy))
        new_s = pandas.Series(withdraws)
        s = s.append(new_s, ignore_index=True)
    all_s[strategy] = s
df = pandas.DataFrame(all_s)
df.head()

In [ ]:
df.mean()

In [ ]:
df.median()

In [ ]:
metrics.cew([Decimal(d) for d in df['vpw']])

In [ ]:
metrics.cew([Decimal(d) for d in df['am']])